# 淘宝用户行为数据分析

## 项目概述
 -使用"taobao100w.csv"数据集（100万条淘宝用户行为记录），完成从SQL练习到Power BI可视化的完整数据分析流程。

## 环境准备
目标：了解数据结构和质量
需要完成：
1. 读取taobao100w.csv文件
2. 查看数据基本信息（shape, columns, dtypes）
3. 检查缺失值情况
4. 检查重复值
5. 检查异常值（如时间戳范围、行为类型范围）

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print('+++++++++++++++++++++++++++++++++++++++++')
df = pd.read_csv('taobao100w.csv')
hang = len(df)
print(f"数据形状:{df.shape}")
print('+++++++++++++++++++++++++++++++++++++++++')
print(f"各列名称:{df.columns.tolist()}")
print('+++++++++++++++++++++++++++++++++++++++++')
datetime=pd.to_datetime(df['timestamp'],unit='s')
print(f"时间范围:{datetime.min()}---{datetime.max()}")
display(df.head())
print('+++++++++++++++++++++++++++++++++++++++++')
# 看数据完整性,看数据类型,看内存占用
print(df.info())
print('+++++++++++++++++++++++++++++++++++++++++')
# 看数值分布,看统计特征,只看数值列
print(df.describe())
print('+++++++++++++++++++++++++++++++++++++++++')
for col in df.columns:
    null_count = df[col].isnull().sum()
    print(f"{col}:缺失值为{null_count:,},缺失率为:{null_count/hang*100}%")
    print('+++++++++++++++++++++++++++++++++++++++++')
duplicate_rows = df.duplicated().sum()
print(f"完全重复的行数:{duplicate_rows:,},重复率为:{duplicate_rows/hang*100}%")

+++++++++++++++++++++++++++++++++++++++++
数据形状:(1000000, 5)
+++++++++++++++++++++++++++++++++++++++++
各列名称:['user_id', 'item_id', 'category_id', 'behavior_type', 'timestamp']
+++++++++++++++++++++++++++++++++++++++++
时间范围:2017-09-11 08:16:39---2017-12-03 16:00:06


,user_id,item_id,category_id,behavior_type,timestamp
0,1,2268318,2520377,pv,1511544070
1,1,2333346,2520771,pv,1511561733
2,1,2576651,149192,pv,1511572885
3,1,3830808,4181361,pv,1511593493
4,1,4365585,2520377,pv,1511596146


+++++++++++++++++++++++++++++++++++++++++
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   user_id        1000000 non-null  int64 
 1   item_id        1000000 non-null  int64 
 2   category_id    1000000 non-null  int64 
 3   behavior_type  1000000 non-null  object
 4   timestamp      1000000 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 38.1+ MB
None
+++++++++++++++++++++++++++++++++++++++++
            user_id       item_id   category_id     timestamp
count  1.000000e+06  1.000000e+06  1.000000e+06  1.000000e+06
mean   4.299565e+05  2.577589e+06  2.706038e+06  1.511962e+09
std    4.353203e+05  1.488887e+06  1.464067e+06  2.305482e+05
min    1.000000e+00  7.200000e+01  2.171000e+03  1.505118e+09
25%    1.091610e+05  1.292238e+06  1.343555e+06  1.511762e+09
50%    1.216220e+05  2.575682e+06  2.693696e+06  1.511965e+09


### 什么是异常值？

- 异常值是指与大多数观测值显著不同的数据点，可能是：
- 数据错误：记录错误、传输错误、设备故障
- 自然异常：真实发生的极端情况
- 数据漂移：系统变更导致的模式变化

#### 时间戳异常
- 检查未来时间戳（不可能大于当前时间）
- 检查过于古老的时间戳
- 检查时间戳为0或极小的值
- 时间顺序异常（同一用户时间倒流）

In [18]:
def check_timestamp_anomalies(df,timestamp_col='timestamp'):
    df['datetime'] = pd.to_datetime(df[timestamp_col],unit='s')
    print("===时间戳分析===")
    print(f"时间范围:{df['datetime'].min()}---{df['datetime'].max()}")
    print(f"跨度:{df['datetime'].max()-df['datetime'].min()}")
    #未来时间戳
    future_mask = df['datetime'] > pd.Timestamp('2017-12-04')# 数据集已知最后时间(活动结束时间)
    future_count = future_mask.sum()
    print(f"\n1. 未来时间戳:{future_count}({future_count/len(df)*100}%)")
    #古早时间戳
    ancient_mask = df['datetime'] < pd.Timestamp('2008-01-01')# 数据集已知最早时间(活动开始时间)
    ancient_count = ancient_mask.sum()
    print(f"\n1. 古早时间戳:{ancient_count}({ancient_count/len(df)*100}%)")
    return {
        'future_timestamps': future_mask,# 未来时间的布尔Series
        'ancient_timestamps': ancient_mask # 古早时间的布尔Series
    }
time_anomalies = check_timestamp_anomalies(df)

===时间戳分析===
时间范围:2017-09-11 08:16:39---2017-12-03 16:00:06
跨度:83 days 07:43:27

1. 未来时间戳:0(0.0%)

1. 古早时间戳:0(0.0%)


In [17]:
# 查看异常时间戳的具体数据
future_data = df[time_anomalies['future_timestamp']]
print(future_data)
ancient_data = df[anomalies['ancient_timestamps']]
print(ancient_data)
# 删除异常数据
df_clean = df[~time_anomalies['future_timestamps'] & ~time_anomalies['ancient_timestamps']]

KeyError: 'future_timestamp'

#### 用户ID和商品ID异常检测

In [7]:
def check_id_anomalies(df,id_columns=['user_id', 'item_id', 'category_id']):
    print("===id检测===")
    anomalies={}
    for col in id_columns:
        if col in df.columns:
            print(f"\n---{col}检查---")

            missing = df[col].isna().sum()
            print(f"缺失值数量为:{missing}({missing/len(df)*100}%)")

            if df[col].dtype == 'object':
                empty = (df[col].str.strip()==' ').sum()
                print(f"空字符串数量: {empty}")

            unique_count = df[col].nunique()
            print(f"3. 唯一值数量: {unique_count}")

            value_counts = df[col].value_counts()

            freq_values = value_counts.values

            Q1 = np.percentile(freq_values,25)
            Q3 = np.percentile(freq_values,75)
            IQR = Q3-Q1
            #取极端的出现次数为异常值,一般常规是1.5倍
            upper_bound = Q3 + 5 * IQR
            high_freq_ids = value_counts[value_counts>upper_bound]
            print(f"4. 高频异常ID数量(>Q3+10*IQR): {len(high_freq_ids)}")

            if len(high_freq_ids)>0:
                for idx,(id_val,count) in enumerate(high_freq_ids.head(10).items()):
                     print(f"     {id_val}: {count}次 ({count/len(df)*100:.2f}%)")
            anomalies[col] = {
                'missing': missing,
                'high_freq_ids': high_freq_ids,
                'value_counts': value_counts
            }
    return anomalies
id_anomalies = check_id_anomalies(df)

===id检测===

---user_id检查---
缺失值数量为:0(0.0%)
3. 唯一值数量: 9739
4. 高频异常ID数量(>Q3+10*IQR): 5
     115477: 781次 (0.08%)
     116139: 724次 (0.07%)
     114912: 689次 (0.07%)
     115906: 668次 (0.07%)
     1010419: 665次 (0.07%)

---item_id检查---
缺失值数量为:0(0.0%)
3. 唯一值数量: 399114
4. 高频异常ID数量(>Q3+10*IQR): 20986
     812879: 304次 (0.03%)
     138964: 232次 (0.02%)
     3845720: 229次 (0.02%)
     2331370: 207次 (0.02%)
     2032668: 206次 (0.02%)
     3708121: 201次 (0.02%)
     1535294: 201次 (0.02%)
     2338453: 196次 (0.02%)
     3031354: 180次 (0.02%)
     4211339: 177次 (0.02%)

---category_id检查---
缺失值数量为:0(0.0%)
3. 唯一值数量: 5796
4. 高频异常ID数量(>Q3+10*IQR): 372
     4756105: 51877次 (5.19%)
     4145813: 33841次 (3.38%)
     2355072: 32571次 (3.26%)
     3607361: 31226次 (3.12%)
     982926: 30797次 (3.08%)
     4801426: 20754次 (2.08%)
     2520377: 20320次 (2.03%)
     1320293: 19264次 (1.93%)
     2465336: 17128次 (1.71%)
     3002561: 15173次 (1.52%)


IQR假设数据近似正态分布，但电商数据通常是：
1. 长尾分布（幂律分布）：少量用户贡献大部分收入
2. 高度偏斜：平均值远大于中位数
3. 包含合法的极端值：VIP用户、大额订单

如果用IQR处理电商数据：
❌ 会把VIP用户标记为异常（业务损失！）
❌ 会删除真实的高价值交易
❌ 会导致分析结果严重偏差
❌ 会让业务方无法信任数据

In [16]:
def detect_business_rule_anomalies(df):
    """
    基于业务规则的异常值检测
    """
    anomalies = {}
    
    # 规则1：价格合理性检查（基于商品类目）
    # 假设我们有商品价格信息
    price_anomalies = detect_price_anomalies(df)
    if len(price_anomalies)>0:
        anomalies['price_anomalies'] = price_anomalies
    # 规则2：购买频率异常
    # 正常用户不会1秒钟内购买10次
    frequency_anomalies = detect_frequency_anomalies(df)
    if len(frequency_anomalies) > 0:  # 判断非空
        anomalies['frequency_anomalies'] = frequency_anomalies
    # 规则3：用户行为模式异常
    pattern_anomalies = detect_pattern_anomalies(df)
    if len(pattern_anomalies) > 0:  # 判断非空
        anomalies['pattern_anomalies'] = pattern_anomalies
    return  anomalies

def detect_price_anomalies(df, category_col='category_id', price_col='price'):
    """
    基于商品类目的价格异常检测
    """
    if price_col not in df.columns or category_col not in df.columns:
        return pd.Series(False, index=df.index)
    
    # 按类目计算价格分布
    category_stats = df.groupby(category_col)[price_col].agg([
        'median', 'mean', 'std', 'count'
    ]).reset_index()
    
    # 计算Z-score，但只在同一类目内比较
    df = df.merge(category_stats, on=category_col, suffixes=('', '_stats'))
    df['price_zscore'] = (df[price_col] - df['mean']) / df['std']
    
    # 标记极端异常（如价格是类目平均的10倍以上）
    price_anomaly_mask = (
        (df['price_zscore'].abs() > 5) |  # 统计异常
        (df[price_col] > df['median'] * 10) |  # 业务异常：价格是同类10倍
        (df[price_col] < 0)  # 逻辑异常：负价格
    )
    
    return price_anomaly_mask

def detect_frequency_anomalies(df, user_col='user_id', timestamp_col='timestamp'):
    """
    检测购买频率异常（可能是刷单或机器人）
    """
    if timestamp_col not in df.columns:
        return pd.Series(False, index=df.index)
    
    # 按用户计算购买频率
    user_stats = df.groupby(user_col).agg({
        timestamp_col: ['count', 'nunique']
    })
    user_stats.columns = ['total_actions', 'unique_days']
    
    # 计算每日平均行为次数
    user_stats['actions_per_day'] = user_stats['total_actions'] / user_stats['unique_days']
    
    # 基于业务经验设定阈值
    # 正常用户：每天购买不超过20次（已经很高了）
    # 异常用户：每天购买超过50次（可能是刷单）
    high_freq_mask = user_stats['actions_per_day'] > 50
    
    return df[user_col].isin(high_freq_mask[high_freq_mask].index)

def detect_pattern_anomalies(df):
    """
    检测用户行为模式异常
    """
    anomalies = []
    
    # 模式1：没有浏览直接购买（可能是API调用或数据错误）
    # 模式2：购买商品后又立即退货（可能异常）
    # 模式3：短时间内跨多个类目高频购买
    
    return pd.Series(False, index=df.index)  # 简化示例

In [18]:
business_rules=detect_business_rule_anomalies(df)
display(business_rules)

{'price_anomalies': 0         False
 1         False
 2         False
 3         False
 4         False
           ...  
 999995    False
 999996    False
 999997    False
 999998    False
 999999    False
 Length: 1000000, dtype: bool,
 'frequency_anomalies': 0         False
 1         False
 2         False
 3         False
 4         False
           ...  
 999995    False
 999996    False
 999997    False
 999998    False
 999999    False
 Name: user_id, Length: 1000000, dtype: bool,
 'pattern_anomalies': 0         False
 1         False
 2         False
 3         False
 4         False
           ...  
 999995    False
 999996    False
 999997    False
 999998    False
 999999    False
 Length: 1000000, dtype: bool}

## 数据分布分析
目标：理解业务数据分布
需要完成：
1. 用户行为类型分布（pv, buy, cart, fav）
2. 活跃用户时间分布（按小时、星期）
3. 热门商品和类目分析
4. 用户购买力分析（基于行为）

### 用户行为类型分布（pv, buy, cart, fav）
 - 统计行为类型分布
 - 按用户分组统计行为类型
 - 计算行为类型比例
 
 综合分析

通过这些分析，你可以获得以下业务洞察：
 - 主要行为类型：了解用户最频繁的行为类型（如浏览、购买、加购、收藏）。
 - 用户行为偏好：分析不同用户的行为偏好，为个性化推荐提供依据。
 - 转化率分析：计算从浏览到购买的转化率，评估营销效果。

In [21]:
# 统计行为类型分布
behavior_distribution = df['behavior_type'].value_counts()
print("用户行为类型分布：")
print(behavior_distribution)

#  按用户分组统计行为类型
user_behavior_distribution = df.groupby('user_id')['behavior_type'].value_counts().unstack().fillna(0)
print("每个用户的行为类型分布：")
print(user_behavior_distribution.head())

# 计算行为类型比例
behavior_distribution_pct = behavior_distribution/behavior_distribution.sum() * 100
print("用户行为类型比例：")
print(behavior_distribution_pct)

pv_count = behavior_distribution.get('pv',0)
buy_count = behavior_distribution.get('buy',0)
if pv_count>0:
    conversion_rate = (buy_count/pv_count)*100
else:
    conversion_rate = 0
print(f'转化率:{conversion_rate:.2f}%')

用户行为类型分布：
behavior_type
pv      896106
cart     55447
fav      28088
buy      20359
Name: count, dtype: int64
每个用户的行为类型分布：
behavior_type   buy  cart   fav     pv
user_id                               
1               0.0   0.0   0.0   55.0
100             8.0   0.0   6.0   84.0
115             0.0   3.0  11.0  227.0
117            10.0   4.0   0.0  156.0
118             0.0   0.0   0.0   90.0
用户行为类型比例：
behavior_type
pv      89.6106
cart     5.5447
fav      2.8088
buy      2.0359
Name: count, dtype: float64
转化率:2.27%


### 活跃用户时间分布（按小时、星期）
 - 转换时间戳
 - 按小时分析活跃用户
 - 按星期分析活跃用户
 - 综合分析
如果需要更详细的分析，可以结合小时和星期进行多级分组：
 
 业务洞察:通过这些分析，你可以获得以下业务洞察：
 - 用户活跃高峰：了解用户最活跃的时间段（如晚上11-14点），可以安排促销活动。
 - 周末与工作日差异：比较周末和工作日的用户行为，调整运营策略。周末更多活跃用户
 - 跨时段趋势：观察不同时间段的用户行为变化，优化用户体验。

In [20]:
print(df['behavior_type'].value_counts())
# 转换时间戳
df['time_data'] = pd.to_datetime(df['timestamp'],unit='s')

df['hour'] = df['time_data'].dt.hour
df['weekday'] = df['time_data'].dt.weekday
df.head()

# 按小时统计活跃用户数
hourly_active_users = df.groupby('hour')['user_id'].nunique()
print("按小时统计的活跃用户数：")
print(hourly_active_users)

# 按星期统计活跃用户数
weekly_active_users = df.groupby('weekday')['user_id'].nunique()
print("按星期统计的活跃用户数：")
print(weekly_active_users)

hour_weekday_active_users = df.groupby(['hour', 'weekday'])['user_id'].nunique().unstack()
print("按小时和星期统计的活跃用户数：")
print(hour_weekday_active_users)

behavior_type
pv      896106
cart     55447
fav      28088
buy      20359
Name: count, dtype: int64
按小时统计的活跃用户数：
hour
0     4370
1     5158
2     5762
3     5815
4     5876
5     6059
6     5879
7     5989
8     5996
9     5780
10    5693
11    6219
12    6560
13    6720
14    6459
15    5060
16    3091
17    1613
18     981
19     700
20     629
21     852
22    1744
23    3250
Name: user_id, dtype: int64
按星期统计的活跃用户数：
weekday
0    7004
1    7020
2    7145
3    7253
4    7731
5    9571
6    9529
Name: user_id, dtype: int64
按小时和星期统计的活跃用户数：
weekday     0     1     2     3     4     5     6
hour                                             
0         722   766   750   767   776  1561  1581
1         911   904   975   947   906  1842  1923
2        1054  1084  1110  1133  1109  2178  2252
3        1077  1029  1086  1127  1123  2203  2279
4        1141  1067  1195  1154  1157  2210  2195
5        1168  1151  1224  1212  1183  2223  2354
6        1076  1099  1163  1146  1153  2209  2278
7    